In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

#Needed for image
import base64





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "QWERqwer"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))
                               
#This droping of object Id was causing a host of issues as "unserialiable JSON Object"
#so this line drops it from the set before it causes these errors
df = df.drop(columns='_id')


#These are the dictionaries for easier searching given the filtering options
#I found regex was easier to use given the differences in the names from entry to entry 
#as opposed to $in which returned stricter results and only somewhat works with regex formating
water_rescue = {"animal_type":"Dog", "breed":{"$regex": 'Labrador|Chesa|NewfoundLand'},"sex_upon_outcome":"Intact Female",
               "age_upon_outcome_in_weeks":{"$gte":26.0}, "age_upon_outcome_in_weeks":{"$lte":156.0},
               "outcome_type":{"$in": ['Rto-Adopt', 'Transfer']}}

wild_rescue = {"animal_type":"Dog", "breed":{"$regex": 'German Shep|Malamute|English Sheep|Siberian|Rott'},"sex_upon_outcome":"Intact Male",
               "age_upon_outcome_in_weeks":{"$gte":26.0}, "age_upon_outcome_in_weeks":{"$lte":156.0},
               "outcome_type":{"$in": ['Rto-Adopt', 'Transfer']}}

indi_rescue = {"animal_type":"Dog", "breed":{"$regex": 'German Shep|Dober|Golden|Bloodhound|Rott'},"sex_upon_outcome":"Intact Male",
               "age_upon_outcome_in_weeks":{"$gte":20.0}, "age_upon_outcome_in_weeks":{"$lte":300.0},
               "outcome_type":{"$in": ['Rto-Adopt', 'Transfer']}}


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('DataTable, Map, Graph')

#DEFAULT TABLE VALUES hidden on launch
default_hidden =[]
for i in df.columns:
    #Excludes values I chose and hides the rest of the columns from user (still can toggle)
    #this was the simpliest way to handle hiding the columns that I could find
    if(i in ["animal_type","breed","age_upon_outcome_in_weeks","name","outcome_type","sex_upon_outcome" ]):
        continue
    else:
        default_hidden.append(i)


#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replaced with image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#I have added the cutomer image to the Header, with my unique identifier for this module
#The image directly links to snhu.edu and works when clicked
    html.Center(style={'display':"flex"}, children=[html.B(html.H1('SNHU CS-340 Project 2_Kovacevich')),
                html.A(href="https://www.snhu.edu", target="_blank" ,children = [html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),
                alt='customer image', width = "100px", height= "100px", )])]),
    html.Hr(),
    #I chose to use a RadioItems object to hold the buttons, this is simple and works consistently for changing the data table
    html.Div(dcc.RadioItems(options = [{"label":"Water Rescue", "value":"Water Rescue", "id":"WaterRescue"}, 
                                       {"label":"Wilderness Rescue", "value":"Wilderness Rescue", "id":"WildRescue"}, 
                                       {"label":"Tracking/Disaster Rescue", "value":"Tracking Disaster Rescue", "id":"IndiRescue"},
                                      {"label":"All", "value":"All", "id":"All"}],
                           id = "radioSelection")
            ),
    html.Hr(),
    dt.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "hideable": True, #allows hiding columns as needed/desired
             "selectable": True,} for i in df.columns
        ],

        data=df.to_dict('records'),
        hidden_columns = default_hidden, #this change is used to hide unnecessary columns at start
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True, #This change allows rows to be selectable
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        style_header ={"minWidth":"150px"} #Stops header from overlapping other column headers
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            #This graph is a pie graph that breaks the animals down by breed
            className='col s12 m6',
            style={"width":"50%"},
            children = [dcc.Graph(id='graph-id')]
            ),
        html.Div(
            #This map allows for the placement of markers based on row selections on datatable
            id='map-id',
            className='col s12 m6',
            style={"width":"50%"},
            children =[dl.Map(style={'width': '800px', 'height': '500px'}, id="map-child",
                              center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])]
            )
        ]),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),
    #This was a simple way to force the window to have space below the map and graph, padding could have been used but this was simplier
    html.P("Thank you for using Grazioso Salvare"),
    html.Br(),
    html.Br(),
])

#############################################
# Interaction Between Components / Controller
#############################################

#This section changes the datatable based off of the users radio selection
@app.callback(Output('datatable-interactivity','data'),
              [Input('radioSelection', 'value')
              ])

def filter_rescue(value):
    #Here a new dataframe is created, then filtered by value and returned(without the unparsable id column)
    df = pd.DataFrame.from_records(shelter.read_all({}))
    
    if value == "Water Rescue":
        df = pd.DataFrame(shelter.read_all(water_rescue))
    if value == "Wilderness Rescue":
        df = pd.DataFrame(shelter.read_all(wild_rescue))
    if value == "Tracking Disaster Rescue":
        df = pd.DataFrame(shelter.read_all(indi_rescue))
    
    
    df = df.drop(columns='_id')
    
    return df.to_dict('records')


#This is used to populate the pie chart
@app.callback(Output('graph-id','figure'),
              [Input('radioSelection', 'value')
              ])

def generate_chart(value):
    #Once again a new dataframe is created, a future improvement would likely be just passing the current data frame
    ddf = pd.DataFrame.from_records(shelter.read_all({}))
    
    if value == "Water Rescue":
        ddf = pd.DataFrame(shelter.read_all(water_rescue))
    if value == "Wilderness Rescue":
        ddf = pd.DataFrame(shelter.read_all(wild_rescue))
    if value == "Tracking Disaster Rescue":
        ddf = pd.DataFrame(shelter.read_all(indi_rescue))
    
    #This creates the pie chart with breed as the differentiating factor
    fig = px.pie(data_frame=ddf, names= ddf['breed'], hole=.2)
    return fig


#Updated Callback for map updates
@app.callback( Output('map-child', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
    Input('datatable-interactivity', "selected_rows")])

def update_map(viewData, selected):

    #This allows a number of selected rows to position markers on the map at the same time
    #for now this shows name and id but could be modified for user needs with relative ease
    dff = pd.DataFrame.from_dict(viewData)

    markers = []
    for i in selected:
        markers.append(dl.Marker(position=[dff['location_lat'][i],dff["location_long"][i]], children=[
                dl.Tooltip(dff["breed"][i]),
                dl.Popup([
                    html.H1(dff["name"][i]),
                    html.H2(dff["animal_id"][i])
                ])
            ]))
    # Austin TX is at [30.75,-97.48]
    return [dl.TileLayer(id="base-layer-id")] + markers

app